In [4]:
# | default_exp stats.dynamic_thresholds

In [5]:
# | export
import numpy as np
import pandas as pd
from scipy.fft import rfft, rfftfreq
from scipy.signal import savgol_filter
from statsmodels.tsa.stattools import acf

In [6]:
# | export


def detect_period_fft(y, fs=1, min_period=2, max_period=None):
    """Estimate dominant period using FFT peak on positive freqs."""
    n = len(y)
    if max_period is None:
        max_period = n // 2
    Y = np.abs(rfft(y - np.mean(y)))
    freqs = rfftfreq(n, d=1 / fs)
    # ignore very low freq (trend) and very high freqs
    valid = (freqs > 0) & (freqs <= 1 / min_period) & (freqs >= 1 / max_period)
    if not np.any(valid):
        return None
    idx = np.argmax(Y[valid])
    chosen = np.where(valid)[0][idx]
    period = 1.0 / freqs[chosen] if freqs[chosen] != 0 else None
    # round to nearest integer
    return int(round(period)) if period is not None else None


def detect_period_acf(y, max_lag=200):
    """Fallback: detect period using ACF peak."""
    acfs = acf(y, nlags=min(max_lag, len(y) // 2), fft=True)
    # ignore lag 0
    peaks = np.where((acfs[1:-1] > acfs[:-2]) & (acfs[1:-1] > acfs[2:]))[0] + 1
    if len(peaks) == 0:
        return None
    return int(peaks[0])


def centered_moving_average(y, k):
    """Centered moving average. k must be odd."""
    return pd.Series(y).rolling(window=k, center=True, min_periods=1).mean().to_numpy()


def seasonal_power_at_period(y, period, fs=1, bandwidth=0.02):
    """Estimate spectral power near seasonal frequency 1/period.
    bandwidth is relative fraction of sampling freq (0..0.5)."""
    n = len(y)
    Y = rfft(y - np.mean(y))
    freqs = rfftfreq(n, d=1 / fs)
    target = 1.0 / period
    # absolute bandwidth in Hz:
    bw = bandwidth
    mask = (freqs >= max(0, target - bw)) & (freqs <= target + bw)
    power = np.sum(np.abs(Y[mask]) ** 2)
    total = np.sum(np.abs(Y) ** 2)
    return power, total


def select_kernel_for_trend(y, max_mult=9, threshold_ratio=0.1, max_k_fraction=0.4):
    n = len(y)
    P = detect_period_fft(y)
    if P is None:
        P = detect_period_acf(y)
    if P is None:
        raise ValueError("Could not detect seasonality period automatically.")
    # Ensure P is at least 2 and not bigger than series
    P = max(2, min(P, n // 2))
    # candidate odd multiples of P: P, 3P, 5P, ...
    candidates = []
    max_k = int(max(3, min(n * max_k_fraction, P * max_mult)))
    m = 1
    while True:
        k = P * (2 * m - 1)  # 1,3,5...
        if k > max_k:
            break
        if k >= 3:  # moving average windows less than 3 are useless for smoothing
            candidates.append(int(k))
        m += 1
    if len(candidates) == 0:
        candidates = [min(n // 3, max(3, P))]
    # original seasonal power
    orig_power, total = seasonal_power_at_period(y, P)
    results = []
    for k in candidates:
        trend = centered_moving_average(y, k)
        resid = y - trend
        power_resid, _ = seasonal_power_at_period(resid, P)
        ratio = power_resid / (orig_power + 1e-12)
        results.append((k, ratio))
        # stop early if good
        if ratio <= threshold_ratio:
            return {
                "selected_k": k,
                "period": P,
                "orig_power": orig_power,
                "resid_power": power_resid,
                "ratio": ratio,
                "candidates": results,
            }
    # if none reached threshold, return candidate with minimal ratio
    best = min(results, key=lambda t: t[1])
    k, ratio = best
    trend = centered_moving_average(y, k)
    resid = y - trend
    power_resid, _ = seasonal_power_at_period(resid, P)
    return {
        "selected_k": k,
        "period": P,
        "orig_power": orig_power,
        "resid_power": power_resid,
        "ratio": ratio,
        "candidates": results,
    }

In [10]:
# | hide
# Example usage
from ts.utils.data import generate_time_series

y = generate_time_series()
out = select_kernel_for_trend(y, threshold_ratio=0.15)
out

{'selected_k': 25,
 'period': 25,
 'orig_power': 250795.09383725695,
 'resid_power': 255354.14047332748,
 'ratio': 1.0181783724965088,
 'candidates': [(25, 1.0181783724965088), (75, 1.031594442175624)]}